# Bayesian Inference

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.integrate import quad
from scipy.stats import beta, binomtest

## Bayesian computation

In [ ]:

# -----------------------------------------------------------------------------
# Toy density (target distribution we want to approximate)
# -----------------------------------------------------------------------------
# In the R code, prop_f was a skew-normal-like distribution.
# We'll define it here directly.

def prop_f(x):
    # skew-normal-like density: proportional to normal * cdf(normal)
    return 2 * norm.pdf(x, loc=0, scale=4) * norm.cdf(5 * x, loc=0, scale=4)

x = np.linspace(-5, 15, 1000)

plt.plot(x, prop_f(x), label="Target density")
plt.ylim(0, 0.3)
plt.title("Toy target density")
plt.legend()
plt.show()

# -----------------------------------------------------------------------------
# Metropolis-Hastings (MCMC)
# -----------------------------------------------------------------------------

def markov_transition(x):
    # Propose new sample from a uniform window around x
    x_new = np.random.uniform(x - 0.1, x + 0.1)
    
    # Acceptance probability (log version for stability)
    p = np.exp(np.log(prop_f(x_new)) - np.log(prop_f(x)))
    
    # Accept/reject
    if np.random.rand() > p:
        x_new = x
    return x_new

np.random.seed(0)
m = 100000  # number of iterations
y = np.zeros(m + 1)

# Run the Markov chain
for i in range(m):
    y[i+1] = markov_transition(y[i])

# Histogram of samples vs. target density
counts, bins = np.histogram(y, bins=50, density=True)
plt.plot(x, prop_f(x), label="Target density")
plt.step(bins[:-1], counts, where="post", color="green", label="MCMC histogram")
plt.ylim(0, 0.3)
plt.legend()
plt.title("Metropolis-Hastings sampling")
plt.show()

# -----------------------------------------------------------------------------
# Laplace approximation
# -----------------------------------------------------------------------------
# Laplace approximation fits a Gaussian around the mode of the log density.

# Objective: negative log of target density
fn = lambda z: -np.log(prop_f(z))

res = minimize(fn, x0=[0], method="L-BFGS-B", hess=True)
mu = res.x[0]
# Extract approximate variance from Hessian (second derivative)
# res.hess_inv is an approximation to the inverse Hessian.
s2 = res.hess_inv.todense()[0,0]

plt.plot(x, prop_f(x), label="Target density")
plt.plot(x, norm.pdf(x, mu, np.sqrt(s2)), "r", label="Laplace approx")
plt.ylim(0, 0.3)
plt.legend()
plt.title("Laplace approximation")
plt.show()

# -----------------------------------------------------------------------------
# Mean-Field Variational Inference (Gaussian variational family)
# -----------------------------------------------------------------------------
# We maximize the ELBO: E_q[log p(x)] - E_q[log q(x)]

# ELBO function to optimize
def elbo(params):
    mu, ss = params  # variational mean and std dev
    
    # log target density
    logp = lambda z: np.log(prop_f(z))
    
    # integrand for E_q[log q(x)]
    def f1(z):
        qz = norm.pdf(z, mu, ss)
        return qz * norm.logpdf(z, mu, ss)
    
    # integrand for E_q[log p(x)]
    def f2(z):
        qz = norm.pdf(z, mu, ss)
        return qz * logp(z)
    
    # Numerical integration over finite domain (approximation)
    eq_logp, _ = quad(f2, -5, 10)
    eq_logq, _ = quad(f1, -5, 10)
    
    return eq_logp - eq_logq

# Optimize ELBO (maximize)
res_vi = minimize(lambda v: -elbo(v), x0=[0,1], bounds=[(None,None),(1e-4,None)], method="L-BFGS-B")
mu_vi, ss_vi = res_vi.x

plt.plot(x, prop_f(x), label="Target density")
plt.plot(x, norm.pdf(x, mu_vi, ss_vi), "r", label="VI approx")
plt.ylim(0, 0.3)
plt.legend()
plt.title("Mean-Field Variational Inference")
plt.show()


### Example
three perspectives on estimating a binomial proportion: frequentist MLE with asymptotic CI, exact binomial CI, and Bayesian Beta posterior credible interval

In [5]:
# -----------------------------------------------------------------------------
# Dataset
# -----------------------------------------------------------------------------
n = 1318   # total trials
k = 41     # number of successes

# Data vector not strictly needed in Python, but equivalent to R's rep()
y = np.concatenate([np.ones(k), np.zeros(n - k)])

# -----------------------------------------------------------------------------
# MLE (sample proportion) and asymptotic normal CI
# -----------------------------------------------------------------------------
p_est = np.mean(y) # MLE estimator for the mean
SE = np.std(y, ddof=1) / np.sqrt(n)  # standard error using sample sd

# Rough 95% CI with ± 2*SE rule of thumb
print(f"Rough 95% CI: {p_est - 2*SE:.4f} - {p_est + 2*SE:.4f}")
print("This CI will contain the true value of the parameter 95% of the time in repeated sampling.")
# but saying that the true value has a 95% probability of being in this interval is incorrect.
print("------------------------------")

print(f"MLE: p = {p_est:.4f} (SE = {SE:.4f})")
print(f"95% CI (asymptotic): [{p_est - 1.96*SE:.4f}, {p_est + 1.96*SE:.4f}]")

# Exact CI via Clopper-Pearson (Beta quantiles)
ci_low = beta.ppf(0.025, k + 1, n - k + 1)
ci_high = beta.ppf(0.975, k + 1, n - k + 1)
print(f"95% CI (exact / Clopper-Pearson): [{ci_low:.4f}, {ci_high:.4f}]")


# -----------------------------------------------------------------------------
# Binomial test
# -----------------------------------------------------------------------------
# Two-sided binomial test for H0: p = 0.5 (default)
# Can change 'p' argument for different nulls
res = binomtest(k, n, p=0.5, alternative='two-sided')
print("\nBinomial test result:")
print(res)

# -----------------------------------------------------------------------------
# Bayesian posterior interval
# -----------------------------------------------------------------------------
# Uniform prior Beta(1,1) -> posterior Beta(k+1, n-k+1)
ci_low_bayes = beta.ppf(0.025, k + 1, n - k + 1)
ci_high_bayes = beta.ppf(0.975, k + 1, n - k + 1)
print(f"Bayesian 95% credible interval (uniform prior): {ci_low_bayes:.4f} - {ci_high_bayes:.4f}")


Rough 95% CI: 0.0215 - 0.0407
MLE: p = 0.0311 (SE = 0.0048)
95% CI (asymptotic): [0.0217, 0.0405]
95% CI (exact / Clopper-Pearson): [0.0230, 0.0419]

Binomial test result:
BinomTestResult(k=41, n=1318, alternative='two-sided', statistic=0.03110773899848255, pvalue=4.75054e-319)
Bayesian 95% credible interval (uniform prior): 0.0230 - 0.0419
